In [2]:
import json

def check_duplicate_attacker_tools(file_path):
    # 用于存储每个 Attacker Tool 出现的次数
    tool_count = {}

    # 读取 JSONL 文件
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                # 解析每行 JSON 数据
                data = json.loads(line)
                # 获取 "Attacker Tool" 名称
                tool_name = data.get("Attacker Tool", None)

                if tool_name:
                    # 统计 "Attacker Tool" 出现的次数
                    if tool_name in tool_count:
                        tool_count[tool_name] += 1
                    else:
                        tool_count[tool_name] = 1
            except json.JSONDecodeError:
                print(f"Invalid JSON line: {line}")

    # 检查是否存在重复的 "Attacker Tool"
    duplicates = {tool: count for tool, count in tool_count.items() if count > 1}

    if duplicates:
        print("Found duplicate Attacker Tools:")
        for tool, count in duplicates.items():
            print(f"{tool}: {count} times")
    else:
        print("No duplicate Attacker Tools found.")

# 指定 JSONL 文件路径
file_path = 'data/all_attack_tools.jsonl'  # 修改为你的文件路径
check_duplicate_attacker_tools(file_path)


No duplicate Attacker Tools found.
